<a href="https://colab.research.google.com/github/RaminTK/PATH/blob/main/PATH05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')


import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
data = pd.read_csv('drive/MyDrive/path/PATHAlerts.csv')
data = data.drop('Video Thumbnail', axis=1)
data = data.drop('Image', axis=1)
data.fillna(0, inplace=True)
data['Content']

0      There's a Red Bull Arena event Sun 07-09-2023 ...
1      Trains from Newark & Harrison are once again d...
2      JSQ-33 via HOB resuming normal service. Earlie...
3      JSQ-33 via HOB delayed. Crew investigating smo...
4      05:20 PM: JSQ-33 via HOB delays continue. Main...
                             ...                        
934    JSQ-33 resuming normal service. PAPD resolved ...
935    NWK-WTC operating with residual delays from an...
936    JSQ-33 delays continue. HOB-33 resuming normal...
937    NWK-WTC operating with  residual delays from a...
938    JSQ-33, HOB-33 delayed. Crew reported a trespa...
Name: Content, Length: 939, dtype: object

In [15]:
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    # Convert to lowercase
    text = text.lower()

    # Remove stopwords (if required, you may need to download stopwords using nltk.download('stopwords'))
    # from nltk.corpus import stopwords
    # stopwords = set(stopwords.words('english'))
    # text = ' '.join(word for word in text.split() if word not in stopwords)

    return text

# Apply the preprocessing function to the 'Content' column
data['Content'] = data['Content'].apply(preprocess_text)
data['Content']

0      there s a red bull arena event sun            ...
1      trains from newark   harrison are once again d...
2      jsq    via hob resuming normal service  earlie...
3      jsq    via hob delayed  crew investigating smo...
4            pm  jsq    via hob delays continue  main...
                             ...                        
934    jsq    resuming normal service  papd resolved ...
935    nwk wtc operating with residual delays from an...
936    jsq    delays continue  hob    resuming normal...
937    nwk wtc operating with  residual delays from a...
938    jsq     hob    delayed  crew reported a trespa...
Name: Content, Length: 939, dtype: object

In [18]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Content'])
X

<939x587 sparse matrix of type '<class 'numpy.int64'>'
	with 20282 stored elements in Compressed Sparse Row format>

In [19]:
kmeans = KMeans(n_clusters=5)  # You can adjust the number of clusters as per your needs
kmeans.fit(X)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=5)

In [20]:
clusters = kmeans.predict(X)

for cluster_id in range(kmeans.n_clusters):
    cluster_tweets = data[clusters == cluster_id]['Content']
    print(f"Cluster {cluster_id}:")
    print(cluster_tweets)
    print()


Cluster 0:
25       st and    st stations closed nightly from ap...
34       st and    st stations closed nightly from ap...
45       st and    st stations closed nightly from ap...
70         st      st      st and    st stations  usu...
78         st      st      st and    st stations  usu...
                             ...                        
898      st and    st stations closed nightly from ap...
912      st and    st stations closed nightly from ap...
925      st and    st stations closed nightly from ap...
926      st and    st stations closed nightly from ap...
931      st and    st stations closed nightly from ap...
Name: Content, Length: 70, dtype: object

Cluster 1:
3      jsq    via hob delayed  crew investigating smo...
4            pm  jsq    via hob delays continue  main...
5            pm  jsq    via hob delayed  dispatch has...
9      nwk wtc delayed  crews working to lock hack ri...
10     nwk wtc delayed  crews continue working to loc...
                        

In [ ]:
import tweepy

# Enter your own Twitter API credentials here
consumer_key = 'API_KEY'
consumer_secret = 'API_SECRET_KEY'
access_token = 'ACCESS_TOKEN'
access_token_secret = 'ACCESS_TOKEN_SECRET'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# You need to replace 'PATHTrain' with the actual Twitter handle of NJ PATH
tweets = api.user_timeline(screen_name='PATHTrain', count=200, tweet_mode='extended')

disruptions = ['delay', 'disruption', 'maintenance', 'suspended', 'cancelled']

for tweet in tweets:
    if any(disruption in tweet.full_text.lower() for disruption in disruptions):
        print(f"{tweet.created_at} - {tweet.full_text}\n")


Unauthorized: ignored